In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Vamos empezar con lo entendimiento dos sin movimiento
- Las columnas SOLD_AMOUNT y SOLD_QUANTITY contienen únicamente valores nulos en este subconjunto. Estos nulos no representan datos faltantes por error, sino que son nulos estructurales, lo que significa que estas ofertas no tuvieron ventas. Esto se confirma porque, en todos los casos, REMAINING_STOCK_AFTER_END es igual a INVOLVED_STOCK, indicando que no hubo movimiento de inventario.
- Este comportamiento se analizará considerando dimensiones como el tiempo de publicación, la duración de la oferta, la categoría del producto (VERTICAL), el dominio agregado (DOM_DOMAIN_AGG1), el tipo de envío (SHIPPING_PAYMENT_TYPE) y el origen (ORIGIN). El objetivo es identificar patrones y posibles factores asociados a la falta de ventas.

In [3]:
df_mov = pd.read_parquet('../../../data/01_processed/ofertas_relampago_sin_ventas_stock_movimentado.parquet')

In [4]:
df_mov

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
0,2021-06-22,2021-06-22 19:00:00+00:00,2021-06-22 23:05:32+00:00,lightning_deal,10,-1,NaN,NaN,None,free_shipping,COMPUTERS,CE,MLM-HEADPHONES
1,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 19:00:01+00:00,lightning_deal,15,7,NaN,NaN,None,free_shipping,ELECTRONICS,CE,MLM-FANS
2,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 21:00:00+00:00,lightning_deal,100,96,NaN,NaN,A,free_shipping,MOBILE,CE,MLM-TABLETS
3,2021-06-22,2021-06-22 07:00:00+00:00,2021-06-22 13:00:04+00:00,lightning_deal,10,5,NaN,NaN,None,none,COMPUTERS,CE,MLM-MICROPHONES
4,2021-06-22,2021-06-22 19:00:00+00:00,2021-06-22 21:22:13+00:00,lightning_deal,15,0,NaN,NaN,None,free_shipping,ELECTRONICS,CE,MLM-MEMORY_CARDS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2021-06-19,2021-06-19 13:00:00+00:00,2021-06-19 21:00:00+00:00,lightning_deal,100,91,NaN,NaN,A,free_shipping,MOBILE,CE,MLM-TABLETS
584,2021-06-19,2021-06-19 14:00:00+00:00,2021-06-19 22:00:00+00:00,lightning_deal,30,29,NaN,NaN,A,free_shipping,MOBILE,CE,MLM-CELLPHONES
585,2021-06-19,2021-06-19 15:00:00+00:00,2021-06-19 23:00:00+00:00,lightning_deal,25,22,NaN,NaN,A,free_shipping,TOOLS AND CONSTRUCTION,HOME & INDUSTRY,MLM-POWER_GRINDERS
586,2021-06-19,2021-06-19 15:00:00+00:00,2021-06-19 23:00:01+00:00,lightning_deal,10,9,NaN,NaN,A,free_shipping,ELECTRONICS,CE,MLM-HAIR_CLIPPERS


In [5]:
df_mov.describe(include= 'all')

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
count,588,588,588,588,588.000000,588.000000,0.0,0.0,235,588,588,588,588
unique,61,244,469,1,NaN,NaN,NaN,NaN,1,2,21,8,93
top,2021-07-12,2021-07-08 17:00:00+00:00,2021-06-01 13:00:08+00:00,lightning_deal,NaN,NaN,NaN,NaN,A,free_shipping,COMPUTERS,CE,MLM-SPEAKERS
freq,29,17,6,588,NaN,NaN,NaN,NaN,235,469,199,408,82
mean,NaN,NaN,NaN,NaN,30.358844,25.287415,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,67.685354,67.464704,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000,-17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,5.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,15.000000,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,20.000000,16.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<b> Interesante que tenemos stock negativado acá, porque eso acontece? </b>
- Vamos hacer un delta de stock para entender cuanto fue movimentado y entender por analysis

In [7]:
# Delta de stock: cuánto se "movió" durante la oferta
df_mov["DELTA_STOCK"] = df_mov["INVOLVED_STOCK"] - df_mov["REMAINING_STOCK_AFTER_END"]

In [9]:
df_mov["OFFER_START_DTTM"] = pd.to_datetime(df_mov["OFFER_START_DTTM"])
df_mov["OFFER_FINISH_DTTM"] = pd.to_datetime(df_mov["OFFER_FINISH_DTTM"])

In [10]:
# Ventana oferta (por conveniencia)
df_mov["offer_window"] = pd.IntervalIndex.from_arrays(df_mov["OFFER_START_DTTM"], df_mov["OFFER_FINISH_DTTM"], closed="both")

In [12]:
checks = []
# 2.1 Fechas válidas
checks.append(("fechas_validas", (df_mov["OFFER_FINISH_DTTM"] >= df_mov["OFFER_START_DTTM"]).all()))

# 2.2 Stock no negativo
checks.append(("stock_no_negativo", (df_mov["INVOLVED_STOCK"] >= 0).all() and (df_mov["REMAINING_STOCK_AFTER_END"] >= 0).all()))

In [14]:
# 2.3 Delta stock consistente con ventas registradas (en este corte, ventas son NaN/0)
# Regla débil: si SOLD_QUANTITY in {NaN, 0} pero DELTA_STOCK > 0 => sospechoso
df_mov["FLAG_SIN_VENTA_CON_MOV"] = ((df_mov["SOLD_QUANTITY"].fillna(0) == 0) & (df_mov["DELTA_STOCK"] > 0))

In [15]:
# 2.4 Duración razonable (p. ej. > 0 minutos y < 30 días)
dur_hours = (df_mov["OFFER_FINISH_DTTM"] - df_mov["OFFER_START_DTTM"]).dt.total_seconds()/3600
df_mov["FLAG_DURACION_RARA"] = (dur_hours <= 0) | (dur_hours > 24*30)

In [16]:
checks.append(("hay_casos_sospechosos", df_mov["FLAG_SIN_VENTA_CON_MOV"].any()))
checks_df = pd.DataFrame(checks, columns=["check","ok"])
checks_df

,check,ok
0,fechas_validas,True
1,stock_no_negativo,False
2,hay_casos_sospechosos,True


In [17]:
sospechosos = df_mov.loc[df_mov["FLAG_SIN_VENTA_CON_MOV"]].copy()
sospechosos["DURACION_HORAS"] = (sospechosos["OFFER_FINISH_DTTM"] - sospechosos["OFFER_START_DTTM"]).dt.total_seconds()/3600

cols_show = [
    "DOMAIN_ID","OFFER_TYPE","VERTICAL","DOM_DOMAIN_AGG1","ORIGIN",
    "OFFER_START_DTTM","OFFER_FINISH_DTTM","INVOLVED_STOCK","REMAINING_STOCK_AFTER_END",
    "DELTA_STOCK","SOLD_QUANTITY","SOLD_AMOUNT","DURACION_HORAS"
]
top_investigar = sospechosos.sort_values(["DELTA_STOCK","DURACION_HORAS"], ascending=[False, True])[cols_show].head(50)
top_investigar.head(20)

,DOMAIN_ID,OFFER_TYPE,VERTICAL,DOM_DOMAIN_AGG1,ORIGIN,OFFER_START_DTTM,OFFER_FINISH_DTTM,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,DELTA_STOCK,SOLD_QUANTITY,SOLD_AMOUNT,DURACION_HORAS
480,MLM-SPEAKERS,lightning_deal,CE,COMPUTERS,A,2021-06-05 13:00:00+00:00,2021-06-05 16:17:06+00:00,40,0,40,NaN,NaN,3.285000
535,MLM-HEADPHONES,lightning_deal,CE,COMPUTERS,A,2021-06-29 14:00:00+00:00,2021-06-29 22:00:00+00:00,100,62,38,NaN,NaN,8.000000
524,MLM-LED_STRIPS,lightning_deal,HOME & INDUSTRY,HOME&DECOR,None,2021-07-12 07:00:00+00:00,2021-07-12 11:25:39+00:00,15,-17,32,NaN,NaN,4.427500
385,MLM-HEADPHONES,lightning_deal,CE,COMPUTERS,A,2021-07-15 14:00:00+00:00,2021-07-15 22:00:00+00:00,200,173,27,NaN,NaN,8.000000
334,MLM-HEADPHONES,lightning_deal,CE,COMPUTERS,A,2021-06-24 15:00:00+00:00,2021-06-24 23:00:00+00:00,100,74,26,NaN,NaN,8.000000
544,MLM-STREAMING_MEDIA_DEVICES,lightning_deal,CE,ELECTRONICS,A,2021-07-26 20:00:00+00:00,2021-07-27 01:00:02+00:00,100,78,22,NaN,NaN,5.000556
205,MLM-TABLETS,lightning_deal,CE,MOBILE,A,2021-07-24 12:00:00+00:00,2021-07-24 20:00:01+00:00,400,379,21,NaN,NaN,8.000278
326,MLM-CELLPHONES,lightning_deal,CE,MOBILE,A,2021-06-17 20:00:00+00:00,2021-06-18 01:00:01+00:00,200,182,18,NaN,NaN,5.000278
224,MLM-SMARTWATCHES,lightning_deal,CE,MOBILE,A,2021-07-10 15:00:00+00:00,2021-07-10 23:00:00+00:00,100,82,18,NaN,NaN,8.000000
68,MLM-SPEAKERS,lightning_deal,CE,COMPUTERS,None,2021-07-23 07:00:00+00:00,2021-07-23 10:24:24+00:00,15,-2,17,NaN,NaN,3.406667


## Si hay casos, y los miramos acá
- Hay casos con REMAINING_STOCK_AFTER_END negativos
- Hay casos que tuvieran 100% de lo involved stock finalizado
- No puedo asumir dado que no tengo toda la informacion de lo dato pero si, seria una cosa a entender porque talvez pueda ser un problema en las ofertas que no estuevieran computando en lo periodo y otros